In [2]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

In [11]:
lineage=pd.read_csv('/Users/DAIR Julia/Desktop/linguistics_capstone/languages.csv')
lineage.drop(['Unnamed: 0.1', 'Unnamed: 0', 'macroarea', 'countries','level', 'category', 'aes'], inplace=True, axis=1)

In [12]:
def split_lineage(x):
    return str(x).split('/')
lineage['lineage']=lineage['lineage'].apply(split_lineage)

In [13]:
#lineage is largely missing, family id is present for all
lineage[lineage['family_id'].notna()]

,name,latitude,longitude,glottocode,family_id,lineage
429,Fulniô Proper,-9.025910,-37.140200,fuln1248,fuln1247,[nan]
430,Yatê,-9.025910,-37.140200,yate1241,fuln1247,[nan]
431,Zoque,17.321619,-94.037312,zoqu1261,mixe1284,[nan]
432,Mixe,16.662186,-94.378501,mixe1286,mixe1284,[nan]
433,Northern North Halmahera,1.091927,127.683556,nort2924,nort2923,[nan]
...,...,...,...,...,...,...
26664,Kitsootso,-5.577220,14.477100,kits1250,atla1278,[nan]
26665,Kindamba,-5.577220,14.477100,kind1243,atla1278,[nan]
26666,Kisibemba,-5.577220,14.477100,kisi1246,atla1278,[nan]
26667,Sira,-1.780840,10.463500,sira1266,atla1278,[nan]


In [24]:
id = lineage['family_id']
ref_fam=pd.merge(left=id, right=lineage, left_on='family_id', right_on='glottocode')
ref_fam.drop_duplicates(subset='family_id_x', ignore_index=True, inplace=True)
ref_fam.drop(labels=['glottocode', 'family_id_y', 'lineage'], inplace=True, axis=1)
ref_fam.to_csv(path_or_buf='/Users/DAIR Julia/Desktop/linguistics_capstone/families.csv', index=False)

In [27]:
lineage['lineage'].value_counts()

[nan]                                                                                                                                                                                                                                     24303
[sino1245, kuki1245, kuki1246, oldk1252]                                                                                                                                                                                                      6
[aust1307, mala1545, cent2237, east2712, ocea1241, cent2060, east2445, poly1242, nucl1485]                                                                                                                                                    6
[aust1307, mala1545, cent2237, east2712, ocea1241, cent2060, east2445, poly1242, nucl1485, nort3246, solo1260, cent2298, east2449, cent2062]                                                                                                  6
[aust1307, mala1545, cent2237, east2712,

In [29]:
class Node:
    def __init__(self, name):
        self.name = name
        self.children = []

def build_hierarchy(lineage):
    root = Node(lineage[0])

    current_node = root

    for name in lineage[1:]:
        child = Node(name)
        current_node.children.append(child)
        current_node = child

    return root

# Create a new column to store the hierarchy for each row
lineage['hierarchy'] = lineage['lineage'].apply(build_hierarchy)


In [30]:
lineage

,name,glottocode,family_id,lineage,hierarchy
0,Fulniô,fuln1247,NaN,[nan],<__main__.Node object at 0x00000276118A1660>
1,Dem,demm1245,NaN,[nan],<__main__.Node object at 0x00000276118A3370>
2,Mixe-Zoque,mixe1284,NaN,[nan],<__main__.Node object at 0x00000276118A1C90>
3,Mure,mure1235,NaN,[nan],<__main__.Node object at 0x00000276118A17B0>
4,North Halmahera,nort2923,NaN,[nan],<__main__.Node object at 0x00000276118A30A0>
...,...,...,...,...,...
26664,Kitsootso,kits1250,atla1278,[nan],<__main__.Node object at 0x0000027614648E80>
26665,Kindamba,kind1243,atla1278,[nan],<__main__.Node object at 0x0000027614648EE0>
26666,Kisibemba,kisi1246,atla1278,[nan],<__main__.Node object at 0x0000027614648F40>
26667,Sira,sira1266,atla1278,[nan],<__main__.Node object at 0x0000027614648FA0>


In [33]:
import networkx as nx
import matplotlib.pyplot as plt

# Select one hierarchy to visualize (e.g., the first row)
hierarchy_to_visualize = lineage['hierarchy'][0]

def plot_hierarchy(node):
    G = nx.Graph()
    queue = [(None, node)]

    while queue:
        parent, current_node = queue.pop()
        G.add_node(current_node.name)
        if parent:
            G.add_edge(parent.name, current_node.name)

        for child in current_node.children:
            queue.append((current_node, child))

    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=3000, node_color='skyblue')
    plt.show()

plot_hierarchy(hierarchy_to_visualize)


TypeError: '_AxesStack' object is not callable